Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

class DoubleGDOptimizer(tf.train.GradientDescentOptimizer):
  def _valid_dtypes(self):
    return set([tf.float32, tf.float64])

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'
save = None
with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [8]:
batch_size = 128
relu_count = 1024
graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_count]))
  biases = tf.Variable(tf.zeros([relu_count]))
  
  weights_hidden = tf.Variable(
    tf.truncated_normal([relu_count, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))  
    
  logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases),weights_hidden) + biases_hidden
  reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)  
  loss = tf.reduce_mean(
tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + 0.01*sum(reg_losses)
  # Optimizer.
#   optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step,100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases),weights_hidden) + biases_hidden)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases),weights_hidden) + biases_hidden)
    
    
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)) #accuracy(predictions2, valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 365.444763
Minibatch accuracy: 16.4%
Validation accuracy: 40.2%
Minibatch loss at step 500: 20.867804
Minibatch accuracy: 80.5%
Validation accuracy: 80.8%
Minibatch loss at step 1000: 8.418095
Minibatch accuracy: 72.7%
Validation accuracy: 80.9%
Minibatch loss at step 1500: 8.420301
Minibatch accuracy: 79.7%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 9.563916
Minibatch accuracy: 78.1%
Validation accuracy: 80.0%
Minibatch loss at step 2500: 5.240034
Minibatch accuracy: 75.8%
Validation accuracy: 82.2%
Minibatch loss at step 3000: 4.376999
Minibatch accuracy: 84.4%
Validation accuracy: 81.8%
Minibatch loss at step 3500: 2.957644
Minibatch accuracy: 81.2%
Validation accuracy: 83.0%
Minibatch loss at step 4000: 2.312643
Minibatch accuracy: 85.9%
Validation accuracy: 81.1%
Minibatch loss at step 4500: 3.197297
Minibatch accuracy: 83.6%
Validation accuracy: 81.1%
Minibatch loss at step 5000: 2.070432
Minibatch accuracy: 85.2%
Validation accu

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [10]:
batch_size = 128
relu_count = 1024
graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_count]))
  biases = tf.Variable(tf.zeros([relu_count]))
  
  weights_hidden = tf.Variable(
    tf.truncated_normal([relu_count, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))  
    
  logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases),weights_hidden) + biases_hidden
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step,100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)  
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases),weights_hidden) + biases_hidden)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases),weights_hidden) + biases_hidden)
    
    
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % 4
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 316.376770
Minibatch accuracy: 8.6%
Validation accuracy: 37.5%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 68.0%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 68.0%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 68.0%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 68.0%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 68.0%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 68.0%
Test accuracy: 74.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [8]:
batch_size = 128
relu_count = 1024
graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  keepprob = tf.placeholder(tf.float32)
  h_fc1_drop = tf.Variable(tf.zeros([relu_count]))
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_count]))
  biases = tf.Variable(tf.zeros([relu_count]))
  
  weights_hidden = tf.Variable(
    tf.truncated_normal([relu_count, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))  
    
  basic_logits = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  h_fc1_drop = tf.nn.dropout(basic_logits, keepprob)                            
  logits = tf.matmul(h_fc1_drop,weights_hidden) + biases_hidden
  reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)  
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + 0.01*sum(reg_losses)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases),weights_hidden) + biases_hidden)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases),weights_hidden) + biases_hidden)
    
    
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keepprob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict={keepprob: 1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keepprob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 452.360931
Minibatch accuracy: 14.8%
Validation accuracy: 28.3%
Minibatch loss at step 500: 50.602859
Minibatch accuracy: 71.1%
Validation accuracy: 80.4%
Minibatch loss at step 1000: 17.669487
Minibatch accuracy: 61.7%
Validation accuracy: 80.1%
Minibatch loss at step 1500: 30.060457
Minibatch accuracy: 68.8%
Validation accuracy: 80.0%
Minibatch loss at step 2000: 16.875278
Minibatch accuracy: 71.9%
Validation accuracy: 78.1%
Minibatch loss at step 2500: 10.415998
Minibatch accuracy: 71.1%
Validation accuracy: 79.1%
Minibatch loss at step 3000: 7.656900
Minibatch accuracy: 72.7%
Validation accuracy: 79.5%
Test accuracy: 87.2%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [16]:
batch_size = 128
hidden_layer1_size = 1024
hidden_layer2_size = 300
hidden_layer3_size = 50
hidden_layer1_stddev = np.sqrt(2.0/784)
hidden_layer2_stddev = np.sqrt(2.0/hidden_layer1_size)
hidden_layer3_stddev = np.sqrt(2.0/hidden_layer2_size)
output_layer_stddev = np.sqrt(2.0/hidden_layer3_size)
hidden_layer1_keepprob = 0.5
hidden_layer2_keepprob = 0.7
hidden_layer3_keepprob = 0.8
beta1 = 1e-5
beta2 = 1e-5
beta3 = 1e-5
beta4 = 1e-5

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    hidden_layer1_weights = tf.Variable(tf.truncated_normal([image_size*image_size, hidden_layer1_size], stddev = hidden_layer1_stddev))
    hidden_layer1_biases = tf.Variable(tf.zeros([hidden_layer1_size]))
    
    hidden_layer2_weights = tf.Variable(tf.truncated_normal([hidden_layer1_size, hidden_layer2_size], stddev = hidden_layer2_stddev))
    hidden_layer2_biases = tf.Variable(tf.zeros([hidden_layer2_size]))
    
    hidden_layer3_weights = tf.Variable(tf.truncated_normal([hidden_layer2_size, hidden_layer3_size], stddev = hidden_layer3_stddev))
    hidden_layer3_biases = tf.Variable(tf.zeros([hidden_layer3_size]))
    
    output_layer_weights = tf.Variable(tf.truncated_normal([hidden_layer3_size, num_labels]))
    output_layer_biases = tf.Variable(tf.zeros([num_labels]))
    
    hidden_layer1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset,hidden_layer1_weights)+hidden_layer1_biases),hidden_layer1_keepprob)
    hidden_layer2 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden_layer1,hidden_layer2_weights)+hidden_layer2_biases),hidden_layer2_keepprob)
    hidden_layer3 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden_layer2,hidden_layer3_weights)+hidden_layer3_biases),hidden_layer3_keepprob)
    output_layer = tf.matmul(hidden_layer3,output_layer_weights) + output_layer_biases
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output_layer,tf_train_labels))
    loss += (beta1*tf.nn.l2_loss(hidden_layer1_weights)+ beta2*tf.nn.l2_loss(hidden_layer2_weights)+ beta3*tf.nn.l2_loss(hidden_layer3_weights)+ beta4*tf.nn.l2_loss(output_layer_weights)) 
    
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 0.4
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 100000, 0.96, staircase=True)
    #learning_rate = 0.1
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    train_prediction = tf.nn.softmax(output_layer)

    validation_layer1 = tf.nn.relu(tf.matmul(tf_valid_dataset,hidden_layer1_weights)+hidden_layer1_biases)
    validation_layer2 = tf.nn.relu(tf.matmul(validation_layer1,hidden_layer2_weights)+ hidden_layer2_biases)
    validation_layer3 = tf.nn.relu(tf.matmul(validation_layer2, hidden_layer3_weights) + hidden_layer3_biases)
    validation_logits = tf.matmul(validation_layer3,output_layer_weights) + output_layer_biases
    valid_prediction = tf.nn.softmax(validation_logits)
    
    test_layer1 = tf.nn.relu(tf.matmul(tf_test_dataset,hidden_layer1_weights)+hidden_layer1_biases)
    test_layer2 = tf.nn.relu(tf.matmul(test_layer1,hidden_layer2_weights)+ hidden_layer2_biases)
    test_layer3 = tf.nn.relu(tf.matmul(test_layer2, hidden_layer3_weights) + hidden_layer3_biases)
    test_logits = tf.matmul(test_layer3,output_layer_weights) + output_layer_biases
    test_prediction = tf.nn.softmax(test_logits)
    
    num_steps = 20000
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        for step in range(num_steps):
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            batch_data = train_dataset[offset:(offset+batch_size), :]
            batch_labels = train_labels[offset:(offset+batch_size), :]
            feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if(step%500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Valid accuracy: %.1f%%" % accuracy(valid_prediction.eval(),valid_labels))        
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(),test_labels))

Minibatch loss at step 0: 5.832546
Minibatch accuracy: 11.7%
Valid accuracy: 17.3%
Minibatch loss at step 500: 0.675570
Minibatch accuracy: 81.2%
Valid accuracy: 83.6%
Minibatch loss at step 1000: 1.002829
Minibatch accuracy: 70.3%
Valid accuracy: 84.8%
Minibatch loss at step 1500: 0.542617
Minibatch accuracy: 82.8%
Valid accuracy: 85.6%
Minibatch loss at step 2000: 0.651992
Minibatch accuracy: 82.8%
Valid accuracy: 85.6%
Minibatch loss at step 2500: 0.497675
Minibatch accuracy: 84.4%
Valid accuracy: 86.5%
Minibatch loss at step 3000: 0.545754
Minibatch accuracy: 85.2%
Valid accuracy: 86.7%
Minibatch loss at step 3500: 0.431292
Minibatch accuracy: 85.2%
Valid accuracy: 87.3%
Minibatch loss at step 4000: 0.470421
Minibatch accuracy: 87.5%
Valid accuracy: 87.4%
Minibatch loss at step 4500: 0.517164
Minibatch accuracy: 84.4%
Valid accuracy: 87.2%
Minibatch loss at step 5000: 0.402450
Minibatch accuracy: 89.8%
Valid accuracy: 87.9%
Minibatch loss at step 5500: 0.487728
Minibatch accuracy: 